In [1]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Optional, List, Dict
from collections import defaultdict
import pandas as pd


# ---------- Keys ----------
@dataclass(frozen=True)
class RaceKey:
    date: str
    race_number: int


@dataclass(frozen=True)
class SecTimeKey:
    date: str
    race_number: int
    horse_id: str


# ---------- Domain models ----------
@dataclass
class RaceInfo:
    class_name: str = ""
    course: str = ""
    date: str = ""
    distance: int = 0
    going: str = ""
    name: str = ""
    prize: float = 0.0
    race_index: int = 0
    race_number: int = 0
    racecourse: str = ""
    rating: str = ""
    sec1_time: Optional[float] = None
    sec2_time: Optional[float] = None
    sec3_time: Optional[float] = None
    sec4_time: Optional[float] = None
    sec5_time: Optional[float] = None
    sec6_time: Optional[float] = None
    track: str = ""
    url: str = ""

    @property
    def key(self) -> RaceKey:
        return RaceKey(self.date, int(self.race_number))


@dataclass()
class RaceResult:
    actual_weight: float = 0.0
    date: str = ""
    declar_horse_wt: float = 0.0
    draw: int = 0
    finish_time: float = 0.0
    horse_id: str = ""
    horse_name: str = ""
    horse_number: int = 0
    jockey: str = ""
    jockey_id: str = ""
    lbw: str = ""
    position: str = ""
    race_index: int = 0
    race_number: int = 0
    running_position: str = ""
    trainer: str = ""
    trainer_id: str = ""
    win_odds: float = 0.0

    @property
    def race_key(self) -> RaceKey:
        return RaceKey(self.date, self.race_number)

    @property
    def sectime_key(self) -> SecTimeKey:
        return SecTimeKey(self.date, self.race_number, self.horse_id)


@dataclass()
class SectionalResult:
    date: str = ""
    horse_code: str = ""
    horse_id: str = ""
    horse_name: str = ""
    horse_number: int = 0
    lbw: str = ""
    position: int = 0
    position_final: str = ""
    race_number: int = 0
    section: int = 0
    subtime1: Optional[float] = None
    subtime2: Optional[float] = None
    time: float = 0.0

    @property
    def sectime_key(self) -> SecTimeKey:
        return SecTimeKey(self.date, self.race_number, self.horse_id)


# ---------- Composition, not inheritance ----------
@dataclass()
class RaceData:
    result: RaceResult
    race: RaceInfo
    sectimes: List[SectionalResult] = field(default_factory=list)


folder = r"data\2023-24\2025-09-27_15-07-49"

races_df = pd.read_csv(f"{folder}/races.csv")
results_df = pd.read_csv(f"{folder}/results.csv")
sectimes_df = pd.read_csv(f"{folder}/sectime.csv")

races: List[RaceInfo] = [RaceInfo(**row) for row in races_df.to_dict(orient="records")]
results: List[RaceResult] = [RaceResult(**row) for row in results_df.to_dict(orient="records")]
sectimes: List[SectionalResult] = [SectionalResult(**row) for row in sectimes_df.to_dict(orient="records")]


# Index races by race key
races_by_key: Dict[RaceKey, RaceInfo] = {r.key: r for r in races}
sections_by_entry: Dict[SecTimeKey, List[SectionalResult]] = defaultdict(list)
for s in sectimes:
    sections_by_entry[s.sectime_key].append(s)

# Optional: index results by entry key
results_by_entry: Dict[RaceKey, RaceResult] = {r.race_key: r for r in results}


sample = results[0]
sample = RaceData(
    sample,
    races_by_key.get(sample.race_key),
    sections_by_entry.get(sample.sectime_key),
)

In [5]:
sample.race.date

'2023-09-10'